In [1]:
# import of adj_mat
# import of nodes_cluster
# import of news_data
# import of crypto_data

# Import section

## adj_mat

## nodes_cluster

## news_data

## crypto_data

# Merging section (probably needed)

# Model definition (probably needed)